# USAU College Nationals Data Prep

In [40]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt

In [41]:
pool = pd.read_csv('./data/ultimate/collegenationals_men/all_poolplay.csv')
pool['pnt_diff'] = df['winner_score'] - df['loser_score']
pool.head()

,year,event,table,Date,Time,home_team,away_team,home_score,away_score,home_seed,away_seed,pnt_diff
0,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,10:30 AM,Colorado (1),Harvard (8),15,11,1,8,4
1,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,10:30 AM,Wisconsin (12),California-San Diego (13),15,7,12,13,8
2,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,2:30 PM,Harvard (8),California-San Diego (13),15,14,8,13,1
3,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,2:30 PM,Wisconsin (12),Eastern Michigan (17),15,11,12,17,4
4,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,4:30 PM,Colorado (1),Eastern Michigan (17),15,9,1,17,6


In [48]:
pool[['home_seed','away_seed','pnt_diff']].groupby(['home_seed','away_seed']).agg(['count','sum'])

pnt_diff    
                       count sum
home_seed away_seed             
1         8                6  23
          12               6  24
          13               6  17
          17               6  21
2         7                6  30
          11               6  20
          14               6  21
          18               6  22
3         6                6  26
          10               6  28
          15               6  37
          19               6  37
4         5                6  13
          9                6  17
          16               6  34
          20               6  33
5         9                6  29
          16               6  19
          20               6  17
6         10               6  34
          15               6  22
          19               6  23
7         11               6  15
          14               6  30
          18               6  27
8         12               6  20
          13               6  15
          17               6  22
9         16               6  33
          20               6  28
10        15               6  29
          19               6  33
11        14               6  26
          18               6  21
12        13               6  29
          17               6  20
13        17               6  28
14        18               6  23
15        19               6  33
16        20               6  25